In [1]:
import pandas as pd

In [2]:
url = "http://www-2.munimadrid.es/CSE6/control/seleccionDatos?numSerie=3050200010"
docs = ["migrations_madrid_04.csv","migrations_madrid_05.csv","migrations_madrid_06.csv","migrations_madrid_07.csv",
        "migrations_madrid_08.csv","migrations_madrid_09.csv","migrations_madrid_10.csv","migrations_madrid_11.csv",
        "migrations_madrid_12.csv","migrations_madrid_13.csv","migrations_madrid_14.csv","migrations_madrid_15.csv",
        "migrations_madrid_16.csv"]
years = [2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]

In [3]:
def parse_migrations_data(file_name):
    data = pd.read_csv("./data/"+file_name, header=1)
    data = data.set_index("Unnamed: 0")
    data = data.filter(regex='Total', axis=0)
    data = data.filter(regex='Total', axis=1)
    data.columns = [ i.split(". ")[-1] for i in data.columns]
    data.index = [ i.split(". ")[-1] for i in data.index]
    data = data.drop(["Total - Ciudad de Madrid"], axis=1)
    data = data.iloc[1:]
    return data

In [4]:
madrid_17 = pd.read_csv("./data/migrations_madrid_17.csv", header=1).dropna()
madrid_17.index = [ str(i).split(". ")[-1] for i in madrid_17["Unnamed: 0"]]
madrid_17 = madrid_17.drop(["Unnamed: 0","Unnamed: 1"], axis=1)
madrid_17 = madrid_17.iloc[1:]

In [5]:
dfs = []
for doc in docs:
    dfs.append(parse_migrations_data(doc))
dfs.append(madrid_17)

In [6]:
dfs_2 = []
for i, year in enumerate(years):
    dfi = dfs[i].stack().reset_index().rename(columns={"level_0":"origin","level_1":"destination",0:"count"})
    for column in dfi.columns:
        dfi[column] = dfi[column].str.replace(",","")
    dfi['year'] = year
    dfs_2.append(dfi)
inner_migrations_madrid = pd.concat(dfs_2)
inner_migrations_madrid["count"] = inner_migrations_madrid["count"].str.replace(".","")

In [ ]:
#districts_lat_long = pd.read_csv("./data/madrid.csv")

In [ ]:
#inner_migrations_madrid = inner_migrations_madrid.merge(districts_lat_long, left_on="origin", 
#                              right_on="district").rename(columns={"lat":"lat_origin", 
#                                "lng":"lng_origin"}).drop(['district'], axis=1)

In [ ]:
#inner_migrations_madrid = inner_migrations_madrid.merge(districts_lat_long, left_on="destination", 
#                              right_on="district").rename(columns={"lat":"lat_destination", 
#                                "lng":"lng_destination"}).drop(['district'], axis=1)

In [ ]:
#inner_migrations_madrid["datetime"] = inner_migrations_madrid["year"].apply(lambda x: datetime.datetime(x,1,1,1,1,0))

In [ ]:
#inner_migrations_madrid_ = inner_migrations_madrid[inner_migrations_madrid["destination"]=="Centro"]

In [7]:
inner_migrations_madrid.to_csv("./data/madrid_internal_migrations.csv", index=False, encoding="utf-8")